## imports and files

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
__file__ = globals()['_dh'][0]
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    PIPELINE_PROFILING_CONFIGS_PATH,
    PIPELINE_PROFILING_RESULTS_STATIC_PATH
)
from experiments.utils.loader import Loader

In [2]:
from json import load

series = 10
experiment_id = 2
config_key_mapper = "key_config_mapper.csv"

series_path = os.path.join(
    PIPELINE_PROFILING_RESULTS_STATIC_PATH,
    'series',
    str(series))

loader = Loader(
    series_path=series_path, config_key_mapper=config_key_mapper, type_of='pipeline')

In [3]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'nodes': [   {   'cpu_request': ['8'],
                     'data_type': 'image',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['yolov5n'],
                     'node_name': 'yolo',
                     'replicas': [1]},
                 {   'cpu_request': ['8'],
                     'data_type': 'text',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['resnet18'],
                     'node_name': 'resnet-human',
                     'replicas': [1]}],
    'pipeline_folder_name': '5-paper-video',
    'pipeline_name': 'paper-video',
    'repetition': 1,
    'series': 10,
    'series_meta': 'sample for making the analysis notebook',
    'timeout': 120,
    'workload_config': {'load_duration': 60, 'loads

## config map reader

In [4]:
key_config_df = loader.key_config_mapper()
display(key_config_df)
key_config_df.keys()

,experiment_id,name,cpu_request_1,memory_request_1,cpu_limit_1,memory_limit_1,model_variant_1,max_batch_size_1,max_batch_time_1,replicas_1,...,cpu_limit_2,memory_limit_2,model_variant_2,max_batch_size_2,max_batch_time_2,replicas_2,load,load_duration,series,series_meta
0,1,paper-video,8,10Gi,8,10Gi,yolov5n,1,1,1,...,8,10Gi,resnet18,1,1,1,1,60,10,sample for making the analysis notebook


Index(['experiment_id', 'name', 'cpu_request_1', 'memory_request_1',
       'cpu_limit_1', 'memory_limit_1', 'model_variant_1', 'max_batch_size_1',
       'max_batch_time_1', 'replicas_1', 'cpu_request_2', 'memory_request_2',
       'cpu_limit_2', 'memory_limit_2', 'model_variant_2', 'max_batch_size_2',
       'max_batch_time_2', 'replicas_2', 'load', 'load_duration', 'series',
       'series_meta'],
      dtype='object')

In [5]:
experiment_results = loader.get_experiment_detail(2)
display(experiment_results)
experiment_results.keys()


,experiment_id,name,cpu_request_1,memory_request_1,cpu_limit_1,memory_limit_1,model_variant_1,max_batch_size_1,max_batch_time_1,replicas_1,...,cpu_limit_2,memory_limit_2,model_variant_2,max_batch_size_2,max_batch_time_2,replicas_2,load,load_duration,series,series_meta


Index(['experiment_id', 'name', 'cpu_request_1', 'memory_request_1',
       'cpu_limit_1', 'memory_limit_1', 'model_variant_1', 'max_batch_size_1',
       'max_batch_time_1', 'replicas_1', 'cpu_request_2', 'memory_request_2',
       'cpu_limit_2', 'memory_limit_2', 'model_variant_2', 'max_batch_size_2',
       'max_batch_time_2', 'replicas_2', 'load', 'load_duration', 'series',
       'series_meta'],
      dtype='object')

In [6]:
filenames = loader.get_result_file_names()
print(filenames)

['1.json']


## Read resutls

In [7]:
results = loader.read_results()
print(results.keys())

dict_keys(['1'])


In [8]:
flattened_results = loader.flatten_results(results['1']['responses'])
len(flattened_results)

60

In [9]:
experiment_id = 1
flattened_results = loader.flatten_results(
    results[str(experiment_id)]['responses'])
latencies, timeout_count = loader.latency_calculator(
    flattened_results)
print(f'{timeout_count=}')
print(pd.DataFrame(latencies))

timeout_count=0
    client_to_model      yolo  yolo_to_resnet-human  resnet-human  \
0           5.508918  0.571432             11.955194      1.145684   
1           8.097746  0.335690              9.745995      0.868405   
2          10.950953  0.297835              8.786453      0.809664   
3          13.335250  0.458232              7.187082      0.912333   
4          26.152968  0.451173             49.698226      0.748294   
5          26.339636  0.345933             50.641995      0.971666   
6          29.451541  0.359898             56.075780      0.761292   
7          32.693781  0.338175             57.300672      0.998475   
8          34.255043  0.446407             56.645904      0.885031   
9          35.334940  0.389381             55.327199      1.089036   
10         38.747419  0.558127             53.408420      1.070539   
11         40.097343  0.355154             52.974384      0.931056   
12         42.873079  0.486978             50.830738      0.856401   
13   

In [10]:
summary = loader.latency_summary(latencies=latencies)
pd.DataFrame(list(summary.items()))

,0,1
0,client_to_model_avg,104.629333
1,client_to_model_p99,184.788449
2,client_to_model_p50,123.149641
3,client_to_model_var,3238.305257
4,client_to_model_max,188.749576
5,client_to_model_min,5.508918
6,yolo_avg,0.313810
7,yolo_p99,0.563582
8,yolo_p50,0.287870
9,yolo_var,0.005420


In [11]:
results = loader.result_processing()
print(results)
results.columns

   experiment_id  client_to_model_avg  client_to_model_p99  \
0              1            104.629333            184.788449   

   client_to_model_p50  client_to_model_var  client_to_model_max  \
0            123.149641           3238.305257            188.749576   

   client_to_model_min  yolo_avg  yolo_p99  yolo_p50  ...  \
0              5.508918   0.31381  0.563582   0.28787  ...   

   resnet-human_pod1_memory_usage_p50  resnet-human_pod1_memory_usage_var  \
0                         713535488.0                        1.527726e+15   

   resnet-human_pod1_memory_usage_max  resnet-human_pod1_memory_usage_min  \
0                         775684096.0                         635478016.0   

   resnet-human_pod1_throughput_avg  resnet-human_pod1_throughput_p99  \
0                           0.11731                          0.277311   

   resnet-human_pod1_throughput_p50  resnet-human_pod1_throughput_var  \
0                          0.116839                          0.004253   

   re

Index(['experiment_id', 'client_to_model_avg', 'client_to_model_p99',
       'client_to_model_p50', 'client_to_model_var', 'client_to_model_max',
       'client_to_model_min', 'yolo_avg', 'yolo_p99', 'yolo_p50',
       ...
       'resnet-human_pod1_memory_usage_p50',
       'resnet-human_pod1_memory_usage_var',
       'resnet-human_pod1_memory_usage_max',
       'resnet-human_pod1_memory_usage_min',
       'resnet-human_pod1_throughput_avg', 'resnet-human_pod1_throughput_p99',
       'resnet-human_pod1_throughput_p50', 'resnet-human_pod1_throughput_var',
       'resnet-human_pod1_throughput_max', 'resnet-human_pod1_throughput_min'],
      dtype='object', length=107)

In [12]:
key_config_df = loader.key_config_mapper()
key_config_df
# selected_key_config_df =  key_config_df[key_config_df['model_variant_1'] == 'yolov5n']
# selected_key_config_df

,experiment_id,name,cpu_request_1,memory_request_1,cpu_limit_1,memory_limit_1,model_variant_1,max_batch_size_1,max_batch_time_1,replicas_1,...,cpu_limit_2,memory_limit_2,model_variant_2,max_batch_size_2,max_batch_time_2,replicas_2,load,load_duration,series,series_meta
0,1,paper-video,8,10Gi,8,10Gi,yolov5n,1,1,1,...,8,10Gi,resnet18,1,1,1,1,60,10,sample for making the analysis notebook


In [13]:
experiment_ids = key_config_df[
    key_config_df['model_variant_1'] == 'yolov5n']['experiment_id'].tolist()
model = 'resnet-human'
pod = 'pod1'
metadata_columns = ['model_variant_1', 'max_batch_size_1']
results_columns = [
    f'{model}_p99',
    f'{model}_{pod}_cpu_usage_count_avg',
    f'{model}_avg']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
display(output)


,model_variant_1,max_batch_size_1,resnet-human_p99,resnet-human_pod1_cpu_usage_count_avg,resnet-human_avg
0,yolov5n,1,1.112262,49.074751,0.857227


In [16]:
# TODO add figures